# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
%load_ext dotenv
%dotenv -o ../01_materials/labs/.env 
# to load the .env file, had to specify it's path
# could have also created a new one in current dir


In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob


PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(PRICE_DATA+"/**/*.parquet/*.parquet")
dd_px = dd.read_parquet(parquet_files)

# dd_px.compute()


# px_path = os.getenv("PRICE_DATA")
# # print(px_path)

# parquet_files = glob(os.path.join(px_path,'*/*.parquet')) 
# # print(type(parquet_files), len(parquet_files))
# print(parquet_files)
# # I had a problem with the following line of code, it is like read method can't handle
# # a list of paths, only a single path. spent almost 3 hours dealing with it, was to no avail
# dd_px_sample = dd.read_parquet(parquet_files).set_index('ticker')
# dd_px_sample.compute()



,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532219,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206848,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712807,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276844,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299923,2819626,Health Care,Life Sciences Tools & Services,2000
...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-11-04,176.639999,179.169998,173.589996,175.179993,175.179993,4164000,Health Care,Pharmaceuticals,2024
ZTS,2024-11-05,174.559998,176.800003,172.250000,175.270004,175.270004,2453800,Health Care,Pharmaceuticals,2024
ZTS,2024-11-06,178.520004,178.919998,169.309998,170.369995,170.369995,5362100,Health Care,Pharmaceuticals,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
dd_rets = (dd_px.groupby('ticker', group_keys=False).apply(
     lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1))
    ).assign(
        returns = lambda x: x['Adj Close']/x['Adj_Close_lag_1'] - 1
    ).assign(
        hi_lo_range = lambda x: x['High'] - x['Low']
    )
                                       )

# dd_rets.compute()



C:\Users\Administrator\AppData\Local\Temp\ipykernel_16360\392928016.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_rets = (dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
HUM,2001-01-02,14.500000,15.250000,14.312500,14.937500,13.374666,933100,Health Care,Managed Health Care,2001,NaN,NaN,NaN,0.937500
HUM,2001-01-03,14.687500,15.625000,14.250000,14.437500,12.926980,1077200,Health Care,Managed Health Care,2001,14.937500,13.374666,-0.033473,1.375000
HUM,2001-01-04,14.375000,14.437500,12.250000,13.187500,11.807763,1217600,Health Care,Managed Health Care,2001,14.437500,12.926980,-0.086580,2.187500
HUM,2001-01-05,13.000000,14.250000,12.750000,13.625000,12.199489,1606100,Health Care,Managed Health Care,2001,13.187500,11.807763,0.033175,1.500000
HUM,2001-01-08,13.625000,14.500000,13.312500,13.562500,12.143527,873700,Health Care,Managed Health Care,2001,13.625000,12.199489,-0.004587,1.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2019-12-24,24.510000,24.510000,24.270000,24.290001,21.141632,714900,Information Technology,Communications Equipment,2019,24.469999,21.298302,-0.007356,0.240000
JNPR,2019-12-26,24.290001,24.530001,24.290001,24.490000,21.315706,1141000,Information Technology,Communications Equipment,2019,24.290001,21.141632,0.008234,0.240000
JNPR,2019-12-27,24.570000,24.610001,24.389999,24.410000,21.246077,3030300,Information Technology,Communications Equipment,2019,24.490000,21.315706,-0.003267,0.220001


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [8]:
df_px = dd_rets.compute()


# df_px['rolling'] = df_px.groupby('ticker')['returns'].apply(
#     lambda x: x.rolling(10).mean()
# )

df_px['rolling'] = df_px.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())

df_px

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,rolling
ticker,,,,,,,,,,,,,,,
HUM,2024-01-02,459.739990,471.549988,458.149994,470.100006,467.762543,1966900,Health Care,Managed Health Care,2024,NaN,NaN,NaN,13.399994,NaN
HUM,2024-01-03,474.910004,475.000000,463.839996,464.519989,462.210266,1307400,Health Care,Managed Health Care,2024,470.100006,467.762543,-0.011870,11.160004,NaN
HUM,2024-01-04,469.100006,471.369995,466.170013,466.540009,464.220245,951800,Health Care,Managed Health Care,2024,464.519989,462.210266,0.004349,5.199982,NaN
HUM,2024-01-05,464.029999,464.750000,453.730011,458.269989,455.991364,1692700,Health Care,Managed Health Care,2024,466.540009,464.220245,-0.017726,11.019989,NaN
HUM,2024-01-08,458.980011,464.380005,451.989990,460.739990,458.449066,1584500,Health Care,Managed Health Care,2024,458.269989,455.991364,0.005390,12.390015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2021-12-27,34.230000,35.160000,34.220001,35.110001,32.587700,2857400,Information Technology,Communications Equipment,2021,34.130001,31.678102,0.028714,0.939999,0.003317
JNPR,2021-12-28,35.099998,35.549999,35.080002,35.430000,32.884716,3582300,Information Technology,Communications Equipment,2021,35.110001,32.587700,0.009114,0.469997,0.004199
JNPR,2021-12-29,35.450001,35.910000,35.349998,35.770000,33.200291,2243800,Information Technology,Communications Equipment,2021,35.430000,32.884716,0.009596,0.560001,0.006599


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?


(1 pt)


even if we don't consider the compute process to transform dask dataframe to pandas, it is still slower than dask.
Dask has the same computational ability and it is faster because it  handles data in chunks and in a binary (the parquet files) and lazy way. for large datasets such as this one 
using pandas over dask is disadvantageous.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.